# Pre Procesameinto Info Lotes

## 1. Carga de librerias

In [1]:
import os
import pandas as pd
import numpy as np

## 2. Definicion de Funciones

In [ ]:
def unir_archivos_csv_carpeta(carpeta):
    archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]
    dfs = []
    
    for archivo in archivos_csv:
        ruta_archivo = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta_archivo)
        dfs.append(df)
    
    dataframe_final = pd.concat(dfs, ignore_index=True)
    return dataframe_final

def dumificar_columna(dataframe, columna):
    dummies = pd.get_dummies(dataframe[columna], prefix=columna, drop_first=True)
    dataframe = pd.concat([dataframe, dummies], axis=1)
    dataframe.drop(columna, axis=1, inplace=True)
    return dataframe

def generar_consecutivos(df, columna_orden, columna_consecutivos):
    # Ordenar el DataFrame según la columna de orden
    df_ordenado = df.sort_values(by=columna_orden)
    
    # Generar valores consecutivos en una nueva columna
    df_ordenado[columna_consecutivos] = range(1, len(df_ordenado) + 1)
    
    return df_ordenado

## 3. Carga de información

In [ ]:
carpeta_archivos = 'C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/igac/REGISTRO_2'

dataframe_final = unir_archivos_csv_carpeta(carpeta_archivos)

In [5]:
dataframe_final.head()

,DEPARTAMENTO,MUNICIPIO,NUMERO_PREDIAL,ZONA_FISICA_1,ZONA_ECONOMICA_1,AREA_TERRENO_1,ZONA_FISICA_2,ZONA_ECONOMICA_2,AREA_TERRENO_2,HABITACIONES_1,...,AREA_CONSTRUIDA_2,HABITACIONES_3,BANOS_3,LOCALES_3,PISOS_3,TIPIFICACION_3,USO_3,PUNTAJE_3,AREA_CONSTRUIDA_3,NUMERO_PREDIAL_ANTERIOR
0,8,8078,080780001000000000001000000000,40,8,210000.00,0,0,0.00,3,...,48.0,0,0,0,0,0,2,60,71.0,8.078e+18
1,8,8078,080780001000000000002000000000,3,6,163876.84,6,6,4897.63,3,...,200.0,0,0,0,0,0,26,40,130.0,8.078e+18
2,8,8078,080780001000000000002000000000,14,6,120923.81,21,6,3220.62,0,...,30.0,0,0,0,0,0,0,0,0.0,8.078e+18
3,8,8078,080780001000000000002000000000,41,8,890321.03,0,0,0.00,0,...,0.0,0,0,0,0,0,0,0,0.0,8.078e+18
4,8,8078,080780001000000000003000000000,3,6,102.00,6,6,163766.00,0,...,0.0,0,0,0,0,0,0,0,0.0,8.078e+18


In [6]:
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
Municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
Municipios=Municipios['Id_Municipio_Dane']
Municipios=Municipios.to_frame()


## 4.Creación Variables Agregadas

In [7]:
# Area Promedio por lote
Area_mean_lote = dataframe_final.groupby('MUNICIPIO')['AREA_TERRENO_1'].mean()
Area_mean_lote=Area_mean_lote.to_frame()
Area_mean_lote = Area_mean_lote.rename(columns={'AREA_TERRENO_1':'Lotes_Mean_Area'})


In [8]:
# Numero de lotes
Numero_lotes = dataframe_final.groupby('MUNICIPIO')['AREA_TERRENO_1'].count()
Numero_lotes=Numero_lotes.to_frame()
Numero_lotes = Numero_lotes.rename(columns={'AREA_TERRENO_1':'Lotes_Cantidad'})


In [9]:
# Area promedio costruida (solo lotes con construcción)

df_filtrado = dataframe_final[dataframe_final['AREA_CONSTRUIDA_1'] > 0]
Area_contruida = df_filtrado.groupby('MUNICIPIO')['AREA_CONSTRUIDA_1'].mean()
Area_contruida=Area_contruida.to_frame()  
Area_contruida = Area_contruida.rename(columns={'AREA_CONSTRUIDA_1':'Lotes_Area_Mean_Construida'})
#Area_contruida

In [10]:
# Numero lotes con construcciónes grandes, medianas, pequeñas,muy pequeños
categorias = pd.qcut(df_filtrado['AREA_CONSTRUIDA_1'], 4, labels=False)
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'AREA_CONSTRUIDA_1':'Lotes_Cat_Area_Construida'})
dataframe_concat = pd.concat([df_filtrado['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Area_Construida', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Area_Construida_' + str(col) for col in nueva_dataframe.columns]
Cat_Area_Construida=nueva_dataframe
#nueva_dataframe = nueva_dataframe.rename(columns={0:'Lotes_Cat_Area_Construida_0'})


In [11]:
# Numero lotes de areas grandes, medianas, pequeñas,muy pequeños
categorias = pd.qcut(dataframe_final['AREA_TERRENO_1'], 4, labels=False)
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'AREA_TERRENO_1':'Lotes_Cat_Area'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Area', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Area_' + str(col) for col in nueva_dataframe.columns]
Cat_Area=nueva_dataframe
#nueva_dataframe = nueva_dataframe.rename(columns={0:'Lotes_Cat_Area_Construida_0'})



In [12]:
# en lotes con info, promedio de numero de habitaciones

df_filtrado = dataframe_final[dataframe_final['HABITACIONES_1'] > 0]
Habitaciones = df_filtrado.groupby('MUNICIPIO')['HABITACIONES_1'].mean()
Habitaciones=Habitaciones.to_frame()  
Habitaciones = Habitaciones.rename(columns={'HABITACIONES_1':'Lotes_Habitacion_promedio'})


In [13]:
# en lotes con info, promedio de numero de baños
df_filtrado = dataframe_final[dataframe_final['BANOS_1'] > 0]
Baños = df_filtrado.groupby('MUNICIPIO')['BANOS_1'].mean()
Baños=Baños.to_frame()  
Baños = Baños.rename(columns={'BANOS_1':'Lotes_Baños_promedio'})


In [14]:
# en lotes con info, promedio de numero de locales

df_filtrado = dataframe_final[dataframe_final['LOCALES_1'] > 0]
Locales = df_filtrado.groupby('MUNICIPIO')['LOCALES_1'].mean()
Locales = Locales.to_frame()  
Locales = Locales.rename(columns={'LOCALES_1':'Lotes_Numero_Locales_promedio'})

In [15]:
# cantidad de lotes por tipo de uso

dataframe_concat=dataframe_final[['MUNICIPIO', 'USO_1']]
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='USO_1', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Tipo_uso_' + str(col) for col in nueva_dataframe.columns]
Tipo_Uso=nueva_dataframe



In [16]:
# cantidad de lotes por tipificación

dataframe_concat=dataframe_final[['MUNICIPIO', 'TIPIFICACION_1']]
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='TIPIFICACION_1', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Tipificacion_' + str(col) for col in nueva_dataframe.columns]
Tipificacion=nueva_dataframe


In [17]:
# porcentaje lotes por categoria area cosntruida

# Calcular la suma por fila
suma_filas = Cat_Area_Construida.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_cat_area_cosntruida = Cat_Area_Construida.div(suma_filas, axis=0) * 100


In [18]:
# porcentaje lotes por categoria area cosntruida

# Calcular la suma por fila
suma_filas = Cat_Area.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_cat_area = Cat_Area.div(suma_filas, axis=0) * 100

In [19]:
# Numero de habitaciones como categoria

df=pd.DataFrame(dataframe_final['HABITACIONES_1'])
df_consecutivos = generar_consecutivos(df, 'HABITACIONES_1', 'HABITACIONES_1_1')
categorias=pd.qcut(df_consecutivos['HABITACIONES_1_1'], 10, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'HABITACIONES_1_1':'Lotes_Cat_Habitacion'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Habitacion', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Habitacion_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Habitacion=nueva_dataframe


In [20]:
# Baños como categoria

df=pd.DataFrame(dataframe_final['BANOS_1'])
df_consecutivos = generar_consecutivos(df, 'BANOS_1', 'BANOS_1_1')
categorias=pd.qcut(df_consecutivos['BANOS_1_1'], 10, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'BANOS_1_1':'Lotes_Cat_Banos'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Banos', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Banos_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Banos=nueva_dataframe


In [21]:
# Numero de locales como categoria

df=pd.DataFrame(dataframe_final['LOCALES_1'])
df_consecutivos = generar_consecutivos(df, 'LOCALES_1', 'LOCALES_1_1')
categorias=pd.qcut(df_consecutivos['LOCALES_1_1'], 4, labels=False).sort_index()
categorias=categorias.to_frame()
categorias = categorias.rename(columns={'LOCALES_1_1':'Lotes_Cat_Locales'})
dataframe_concat = pd.concat([dataframe_final['MUNICIPIO'], categorias], axis=1)
nueva_dataframe = pd.pivot_table(dataframe_concat, index=dataframe_concat.MUNICIPIO, columns='Lotes_Cat_Locales', aggfunc='size', fill_value=0)
nueva_dataframe.columns = ['Lotes_Cat_Locales_' + str(col) for col in nueva_dataframe.columns]
Cat_Lotes_Locales=nueva_dataframe


In [22]:
# porcentaje lotes por categoria de habitacion

# Calcular la suma por fila
suma_filas = Cat_Lotes_Banos.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Habitacion = Cat_Lotes_Banos.div(suma_filas, axis=0) * 100

In [23]:
# porcentaje lotes por categoria de hbaitacion baños

# Calcular la suma por fila
suma_filas = Cat_Lotes_Banos.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Banos = Cat_Lotes_Banos.div(suma_filas, axis=0) * 100


In [24]:
# porcentaje lotes por categoria de locales

# Calcular la suma por fila
suma_filas = Cat_Lotes_Locales.sum(axis=1)

# Calcular los porcentajes de cada valor en las columnas
porcentajes_Cat_Lotes_Locales = Cat_Lotes_Locales.div(suma_filas, axis=0) * 100

## 5. Integración de Datos

In [25]:
df_unido=Municipios.merge(Area_mean_lote, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Numero_lotes, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Area_contruida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Area_Construida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Area, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Habitaciones, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Baños, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Tipo_Uso, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Tipificacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_cat_area_cosntruida, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_cat_area, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Habitacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Banos, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(Cat_Lotes_Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Habitacion, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Banos, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')
df_unido=df_unido.merge(porcentajes_Cat_Lotes_Locales, left_on='Id_Municipio_Dane', right_on='MUNICIPIO', how='left')

In [26]:
shape = df_unido.shape
  
# Printing Number of columns
print('Number of columns :', shape[1])
#df_unido

Number of columns : 159


In [29]:
df_unido.iloc[[200, 250, 300, 350, 400, 450, 600, 550, 700, 650]]

,Id_Municipio_Dane,Lotes_Mean_Area,Lotes_Cantidad,Lotes_Area_Mean_Construida,Lotes_Cat_Area_Construida_0_x,Lotes_Cat_Area_Construida_1_x,Lotes_Cat_Area_Construida_2_x,Lotes_Cat_Area_Construida_3_x,Lotes_Cat_Area_0_x,Lotes_Cat_Area_1_x,...,Lotes_Cat_Banos_4,Lotes_Cat_Banos_5,Lotes_Cat_Banos_6,Lotes_Cat_Banos_7,Lotes_Cat_Banos_8,Lotes_Cat_Banos_9,Lotes_Cat_Locales_0_y,Lotes_Cat_Locales_1_y,Lotes_Cat_Locales_2_y,Lotes_Cat_Locales_3_y
200,15090,3.016038e+04,1356.0,67.545126,193.0,197.0,89.0,75.0,116.0,124.0,...,81.637168,0.000000,0.000000,16.740413,0.000000,1.622419,0.00000,0.0,99.631268,0.368732
250,15466,8.831638e+03,6994.0,87.441215,456.0,358.0,290.0,410.0,207.0,660.0,...,60.937947,27.866743,8.521590,0.000000,0.000000,2.673720,0.00000,0.0,98.856162,1.143838
300,15790,2.009516e+04,6900.0,86.202774,943.0,628.0,719.0,846.0,656.0,452.0,...,70.840580,0.000000,0.000000,0.000000,25.275362,3.884058,0.00000,0.0,99.405797,0.594203
350,18247,6.296452e+04,8389.0,89.136676,914.0,1153.0,1457.0,1283.0,1126.0,3592.0,...,0.000000,65.871975,0.000000,0.000000,31.422100,2.705924,0.00000,0.0,97.377518,2.622482
400,19821,2.512838e+05,1158.0,363.928783,188.0,142.0,163.0,181.0,279.0,353.0,...,0.000000,0.000000,65.284974,28.670121,0.000000,6.044905,0.00000,0.0,0.000000,100.000000
450,23672,1.279280e+04,8895.0,67.819818,2415.0,1204.0,1068.0,911.0,1299.0,1460.0,...,0.000000,0.000000,0.000000,0.000000,15.874087,4.890388,0.00000,0.0,0.000000,100.000000
500,25328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,25781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,27745,5.510786e+06,307.0,82.962406,39.0,44.0,30.0,20.0,233.0,39.0,...,0.000000,0.000000,0.000000,0.000000,32.247557,3.583062,0.00000,0.0,0.000000,100.000000
650,44430,3.195430e+04,31859.0,104.296769,5283.0,3667.0,5307.0,6016.0,6800.0,12198.0,...,0.000000,0.000000,0.000000,32.060014,0.000000,8.917417,96.16121,0.0,0.000000,3.838790


In [64]:
df_unido.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/Lotes.csv', index=False)